# 00 - Preparations

In [ ]:
from owslib.wms import WebMapService
from types import SimpleNamespace
from pathlib import Path
from tqdm import tqdm

In [ ]:
CFG = SimpleNamespace()
CFG.WMS_URL = 'http://geoservices.buergernetz.bz.it/mapproxy/root/ows?'

# 01 - Explore WMS service (only for testing)

https://owslib.readthedocs.io/en/latest/usage.html#

### Find out what a WMS has to offer (service metadata)

In [ ]:
wms = WebMapService(CFG.WMS_URL)
wms.identification.type

In [ ]:
wms.identification.version

In [ ]:
wms.identification.title

In [ ]:
wms.identification.abstract

### Available layers

In [ ]:
list(wms.contents)

In [ ]:
# Select the layer for future use
CFG.LAYER_NAME = 'p_bz-Orthoimagery:Aerial-2023-RGB'

### Details of a layer

In [ ]:
wms[CFG.LAYER_NAME].title

In [ ]:
wms[CFG.LAYER_NAME].queryable

In [ ]:
wms[CFG.LAYER_NAME].opaque

In [ ]:
wms[CFG.LAYER_NAME].boundingBox

In [ ]:
wms[CFG.LAYER_NAME].boundingBoxWGS84

In [ ]:
wms[CFG.LAYER_NAME].crsOptions

In [ ]:
wms[CFG.LAYER_NAME].styles

### Available methods, their URLs, and available formats

In [ ]:
[op.name for op in wms.operations]

In [ ]:
wms.getOperationByName('GetMap').methods

In [ ]:
wms.getOperationByName('GetMap').formatOptions

### Make a request for imagery

In [ ]:
img = wms.getmap(
    layers=[CFG.LAYER_NAME],
    #styles=['visual_bright'],
    srs='EPSG:3857', # Web Mercator
    bbox=(1242657, 5878915, 1242757, 5879015),
    size=(640, 640),
    format='image/jpeg',
    transparent=True,
    )

out = open('test_image.jpg', 'wb')
out.write(img.read())
out.close()

# 02 - Download images from WMS

## Function to load images from WMS

In [ ]:
def load_and_save_images_from_wms(
        url: str,
        layer_name: str,
        coords_center: tuple,
        img_size: tuple,
        resolution: float,
        path: Path,
        filename: str,
        ):
    """
    Load and save images from a WMS server based on given coordinates and resolution.
    Parameters:
    url (str): The WMS server URL.
    coords_center (tuple): The x and y coordinates of the center point.
    img_size (tuple): The size of the image (width, height) in pixels.
    resolution (float): The resolution in meters per pixel.
    """

    wms = WebMapService(url)

    x_min = coords_center[0] - (img_size[0] * resolution) / 2
    x_max = coords_center[0] + (img_size[0] * resolution) / 2
    y_min = coords_center[1] - (img_size[1] * resolution) / 2
    y_max = coords_center[1] + (img_size[1] * resolution) / 2

    img = wms.getmap(
        layers=[layer_name],
        srs='EPSG:3857', # Web Mercator
        bbox=(x_min, y_min, x_max, y_max),
        size=img_size,
        format='image/jpeg',
        transparent=True,
        )

    out = open(path / f'{filename}.jpg', 'wb')
    out.write(img.read())
    out.close()

In [ ]:
CFG.LAYER_NAME = 'p_bz-Orthoimagery:Aerial-2023-RGB'
CFG.IMG_SIZE = (640, 640)
CFG.RESOLUTION = 0.2
CFG.PATH = Path().cwd() / 'raw_images_640x640'

In [ ]:
load_and_save_images_from_wms(
    url=CFG.WMS_URL,
    layer_name=CFG.LAYER_NAME,
    coords_center=(1242660, 5878955),
    img_size=CFG.IMG_SIZE,
    resolution=CFG.RESOLUTION,
    path=CFG.PATH,
    filename='0',
    )

### Download the images in a specified grid

In [ ]:
# Set the raster size and center
CFG.RASTER_CENTER = (1173975, 5891150)
CFG.RASTER_SIZE = (3, 3)

x_step = CFG.IMG_SIZE[0] * CFG.RESOLUTION
y_step = CFG.IMG_SIZE[1] * CFG.RESOLUTION

for col_x in tqdm(range(CFG.RASTER_SIZE[0]), desc='Column'):
    for row_y in range(CFG.RASTER_SIZE[1]):

        # Calculate the coordinates for the center of the image
        coords_center = (
            CFG.RASTER_CENTER[0] + col_x * x_step,
            CFG.RASTER_CENTER[1] + row_y * y_step,
        )

        # Load and save the image
        filename = f'{int(coords_center[0])}_{int(coords_center[1])}'
        load_and_save_images_from_wms(
            url=CFG.WMS_URL,
            layer_name=CFG.LAYER_NAME,
            coords_center=coords_center,
            img_size=CFG.IMG_SIZE,
            resolution=CFG.RESOLUTION,
            path=CFG.PATH,
            filename=filename,
            )

CFG

# 03 - Notes

### Loaded images

- p_bz-Orthoimagery:Aerial-2023-RGB
    - Lana, Boznerstraße - CFG.RASTER_CENTER = (1242660, 5878955), CFG.RASTER_SIZE = (5, 5)
    - Gargazon, Gemeindeplatz - RASTER_CENTER = (1247000, 5874540), RASTER_SIZE = (5, 5)
    - Eppan, Montigglerstraße - RASTER_CENTER = (1254790, 5852700), RASTER_SIZE = (5, 5)
    - Niederolang, Oberrain - RASTER_CENTER = (1338270, 5903870), RASTER_SIZE = (5, 5)
    - Mals, Bahnhofstraße - RASTER_CENTER = (1173975, 5891150), RASTER_SIZE = (5, 5)